In [4]:
import pandas as pd
import datetime as dt
import wrds

conn = wrds.Connection(wrds_username='bosen', wrds_password='Y@QC8f4qEjCz!eY')


Loading library list...
Done


In [5]:
query = """ SELECT * FROM ciq.wrds_gvkey """
gvkey = conn.raw_sql(query)

gvkey['startdate'] = pd.to_datetime(gvkey['startdate'], errors='coerce')
gvkey['enddate'] = pd.to_datetime(gvkey['enddate'], errors='coerce')

gvkey['startdate'] = gvkey['startdate'].fillna(pd.Timestamp("1901-01-01"))
gvkey['enddate'] = gvkey['enddate'].fillna(pd.Timestamp("2261-12-31"))  # use the maximum timestamp instead of 2999-12-31

In [6]:
query = """
SELECT company_id as companyid, entity_pname, ratingdate, ratingsymbol, ratingactionword, unsol
FROM ciq_ratings.wrds_erating
WHERE longtermflag = 1 AND ratingtypename = 'Local Currency LT' AND ratingdate >= '1990-01-01'
"""
ratings = conn.raw_sql(query)

ratings['ratingdate'] = pd.to_datetime(ratings['ratingdate'], errors='coerce')

symbols = ['AAA', 'AA+', 'AA', 'AA-', 'A+', 'A', 'A-', 'BBB+', 'BBB', 'BBB-',
           'BB+', 'BB', 'BB-', 'B+', 'B', 'B-', 'CCC+', 'CCC', 'CCC-', 'CC',
           'C', 'D', 'SD', 'NR', 'R']
ratings = ratings[ratings['ratingsymbol'].isin(symbols)]

print(f'\nThe number of rows in ratings: {len(ratings):,}.\n')


The number of rows in ratings: 239,212.



If we exclude ratings that does not happened within the time window of "start date" and "end date", we will lose up to 56 D ratings. I do not think it is acceptable. But we will continue with that.

In [7]:
excluded_ratings = ratings.merge(gvkey[['gvkey', 'companyid', 'startdate', 'enddate']], on='companyid', how='inner')

excluded_ratings = excluded_ratings[
    (excluded_ratings['ratingdate'] < excluded_ratings['startdate']) |
    (excluded_ratings['ratingdate'] > excluded_ratings['enddate'])
]

excluded_rating_counts = excluded_ratings['ratingsymbol'].value_counts()
excluded_D_count = excluded_rating_counts.get('D', 0)

print('Excluded rating counts:')
print(excluded_rating_counts)
print(f"Number of excluded 'D' ratings: {excluded_D_count}")


Excluded rating counts:
ratingsymbol
BBB+    331
B+      328
BB-     311
BBB     301
B       292
A-      271
NR      250
A       235
BBB-    217
BB      193
BB+     185
B-      155
A+      124
AA-      87
CCC+     71
D        56
CCC      46
AA       42
CC       34
SD       23
CCC-     20
AA+      17
AAA      14
R         1
Name: count, dtype: Int64
Number of excluded 'D' ratings: 56


In [8]:
gvkey = gvkey[gvkey['companyid'].isin(ratings['companyid'].unique())]

ratings2 = pd.merge(gvkey[['gvkey', 'companyid', 'startdate', 'enddate']], 
                    ratings, 
                    on='companyid')

# restrict the ratings to the date range of the company here
ratings2 = ratings2[(ratings2['ratingdate'] >= ratings2['startdate']) & 
                    (ratings2['ratingdate'] <= ratings2['enddate'])]

print(f'\nThe number of rows after merging with date filter: {len(ratings2):,}.\n')



The number of rows after merging with date filter: 86,982.



In [9]:
ratings3 = ratings2.drop_duplicates(subset=['gvkey', 'ratingdate'])
ratings3

,gvkey,companyid,startdate,enddate,entity_pname,ratingdate,ratingsymbol,ratingactionword,unsol
0,210835,18511.0,1901-01-01,2261-12-31,3i Group PLC,1995-12-07,AA-,New Rating,N
2,210835,18511.0,1901-01-01,2261-12-31,3i Group PLC,2001-12-20,A+,Downgrade,N
5,210835,18511.0,1901-01-01,2261-12-31,3i Group PLC,2008-11-11,A,Downgrade,N
6,210835,18511.0,1901-01-01,2261-12-31,3i Group PLC,2009-01-28,A-,Downgrade,N
7,210835,18511.0,1901-01-01,2261-12-31,3i Group PLC,2009-02-23,BBB+,Downgrade,N
...,...,...,...,...,...,...,...,...,...
90579,050852,1871267685.0,1901-01-01,2261-12-31,Magnera Corporation,2022-10-26,CCC+,Downgrade,N
90582,050852,1871267685.0,1901-01-01,2261-12-31,Magnera Corporation,2024-09-26,B+,Upgrade,Y
90583,045167,1871945009.0,1901-01-01,2261-12-31,Solventum Corporation,2024-02-20,BBB-,New Rating,Y
90584,050720,1878370658.0,1901-01-01,2261-12-31,South Bow Corporation,2024-08-12,BBB-,New Rating,Y


We check the possibility of one `gvkey` correspond to several `companyid` here.

In [36]:
check = ratings3.groupby('gvkey')['companyid'].nunique()
selected_gvkeys = check[check > 1].index
duplicate_data = ratings3[ratings3['gvkey'].isin(selected_gvkeys)]
duplicate_data =duplicate_data.sort_values(['gvkey', 'ratingdate'], ascending = [True, True])

duplicate_data

,gvkey,companyid,startdate,enddate,entity_pname,ratingdate,ratingsymbol,ratingactionword,unsol
58103,001081,932610.0,1997-05-31,2000-04-18,Stone Consolidated Corp.,1997-06-10,NR,Not Rated,N
42230,001081,527596.0,2000-04-19,2261-12-31,Donohue Inc.,2000-04-20,BBB-,Downgrade,N
42235,001081,527596.0,2000-04-19,2261-12-31,Abitibi-Consolidated Inc.,2003-02-19,BB+,Downgrade,N
42231,001081,527596.0,2000-04-19,2261-12-31,Donohue Inc.,2003-03-14,NR,Not Rated,N
42237,001081,527596.0,2000-04-19,2261-12-31,Abitibi-Consolidated Inc.,2004-03-04,BB,Downgrade,N
...,...,...,...,...,...,...,...,...,...
89598,264397,332169097.0,1901-01-01,2016-06-30,Cardtronics Inc.,2011-07-28,BB,Upgrade,N
89599,264397,332169097.0,1901-01-01,2016-06-30,Cardtronics Inc.,2013-06-27,BB+,Upgrade,N
60395,264397,1533412.0,2016-07-01,2261-12-31,Cardtronics plc,2017-03-30,BB+,New Rating,N
60396,264397,1533412.0,2016-07-01,2261-12-31,Cardtronics plc,2018-02-26,BB,Downgrade,N


In [ ]:
# It is good to set the index to the primary keys for the data, but I still want to use them as columns

ratings4_with_primarykeys = ratings3.set_index(['gvkey', 'ratingdate'])
ratings4_with_primarykeys

companyid  startdate    enddate  \
gvkey  ratingdate                                       
210835 1995-12-07       18511.0 1901-01-01 2261-12-31   
       2001-12-20       18511.0 1901-01-01 2261-12-31   
       2008-11-11       18511.0 1901-01-01 2261-12-31   
       2009-01-28       18511.0 1901-01-01 2261-12-31   
       2009-02-23       18511.0 1901-01-01 2261-12-31   
...                         ...        ...        ...   
050852 2022-10-26  1871267685.0 1901-01-01 2261-12-31   
       2024-09-26  1871267685.0 1901-01-01 2261-12-31   
045167 2024-02-20  1871945009.0 1901-01-01 2261-12-31   
050720 2024-08-12  1878370658.0 1901-01-01 2261-12-31   
050345 2024-06-17  1884847541.0 1901-01-01 2261-12-31   

                                           entity_pname ratingsymbol  \
gvkey  ratingdate                                                      
210835 1995-12-07                          3i Group PLC          AA-   
       2001-12-20                          3i Group PLC           A+   
       2008-11-11                          3i Group PLC            A   
       2009-01-28                          3i Group PLC           A-   
       2009-02-23                          3i Group PLC         BBB+   
...                                                 ...          ...   
050852 2022-10-26                   Magnera Corporation         CCC+   
       2024-09-26                   Magnera Corporation           B+   
045167 2024-02-20                 Solventum Corporation         BBB-   
050720 2024-08-12                 South Bow Corporation         BBB-   
050345 2024-06-17  Concentra Group Holdings Parent, Inc          BB-   

                  ratingactionword unsol  
gvkey  ratingdate                         
210835 1995-12-07       New Rating     N  
       2001-12-20        Downgrade     N  
       2008-11-11        Downgrade     N  
       2009-01-28        Downgrade     N  
       2009-02-23        Downgrade     N  
...                            ...   ...  
050852 2022-10-26        Downgrade     N  
       2024-09-26          Upgrade     Y  
045167 2024-02-20       New Rating     Y  
050720 2024-08-12       New Rating     Y  
050345 2024-06-17       New Rating     Y  

[44692 rows x 7 columns]

In [12]:
# Example 1
specific_gvkey = '001045'  
specific_ratingdate = '2011-11-29' 
specific_row = ratings4_with_primarykeys.loc[(specific_gvkey, specific_ratingdate)].to_frame().T

specific_row

,,companyid,startdate,enddate,entity_pname,ratingsymbol,ratingactionword,unsol
001045,2011-11-29,168569.0,1901-01-01 00:00:00,2261-12-31 00:00:00,American Airlines Group Inc.,D,Downgrade,N


In [13]:
# Example 2
specific_gvkey = '001045'
specific_rows = ratings4_with_primarykeys.loc[specific_gvkey]
specific_rows

,companyid,startdate,enddate,entity_pname,ratingsymbol,ratingactionword,unsol
ratingdate,,,,,,,
1990-08-20,168569.0,1901-01-01,2261-12-31,American Airlines Group Inc.,A-,Downgrade,N
1991-04-05,168569.0,1901-01-01,2261-12-31,American Airlines Group Inc.,BBB+,Downgrade,N
1992-06-01,168569.0,1901-01-01,2261-12-31,American Airlines Group Inc.,BBB,Downgrade,N
1993-03-11,168569.0,1901-01-01,2261-12-31,American Airlines Group Inc.,BB+,<NA>,N
1997-06-16,168569.0,1901-01-01,2261-12-31,American Airlines Group Inc.,BBB-,<NA>,N
2001-09-20,168569.0,1901-01-01,2261-12-31,American Airlines Group Inc.,BB,Downgrade,N
2002-06-28,168569.0,1901-01-01,2261-12-31,American Airlines Group Inc.,BB-,Downgrade,N
2003-02-28,168569.0,1901-01-01,2261-12-31,American Airlines Group Inc.,B-,Downgrade,N
2003-03-28,168569.0,1901-01-01,2261-12-31,American Airlines Group Inc.,CCC,Downgrade,N


In [37]:
ratings = ratings3

In [38]:
defaults = ratings[ratings.ratingsymbol.isin(['D', 'SD', 'R'])]
defaults # some companies have multiple default ratings

,gvkey,companyid,startdate,enddate,entity_pname,ratingdate,ratingsymbol,ratingactionword,unsol
75,065345,18759.0,1901-01-01,2261-12-31,American Capital Ltd.,2010-06-28,SD,Downgrade,N
208,126136,19609.0,1901-01-01,2261-12-31,Charter Communications Inc.,2006-09-15,SD,Downgrade,N
214,126136,19609.0,1901-01-01,2261-12-31,Charter Communications Inc.,2009-01-15,D,Downgrade,N
216,126136,19609.0,1901-01-01,2261-12-31,Charter Communications Inc.,2009-03-27,D,Downgrade,N
377,060839,21219.0,1901-01-01,2261-12-31,J.B. Poindexter & Co. Inc.,2003-06-13,D,Downgrade,N
...,...,...,...,...,...,...,...,...,...
90198,034029,578134665.0,1901-01-01,2261-12-31,Farfetch Ltd.,2024-05-10,D,Downgrade,N
90205,034051,578432369.0,1901-01-01,2261-12-31,Garrett Motion Inc.,2020-09-24,D,Downgrade,N
90348,334759,629456206.0,1901-01-01,2261-12-31,Sinic Holdings (Group) Co. Ltd.,2021-10-19,SD,Downgrade,N
90496,039962,714706835.0,1901-01-01,2261-12-31,Loyalty Ventures Inc.,2023-03-10,D,Downgrade,N


In [ ]:
# ratings.to_csv('ratings.csv')

## Financial Data

In [16]:
# this is where we pull down the financials for use in modeling
# we should add anything additional that we can think of

sql_financials = """
SELECT 
    gvkey, 
    datadate,          -- Statement Date (Fiscal Year-End)
    fyear,             -- Fiscal Year
    fyr,               -- Fiscal Year-End Month
    at,                -- Total Assets
    lt,                -- Total Liabilities
    ceq,               -- Common Equity
    act,               -- Current Assets
    lct,               -- Current Liabilities
    invt,              -- Inventories
    rect,              -- Accounts Receivable
    ap,                -- Accounts Payable
    dlc,               -- Debt in Current Liabilities (Short-Term Debt)
    dltt,              -- Debt in Long-Term Liabilities
    dltis,             -- Debt Issued
    dvt,               -- Debt Valuation
    che,               -- Cash and Short-Term Investments
    xint,              -- Interest Expense
    xrd,               -- Research and Development Expense
    xsga,              -- Selling, General, and Administrative Expenses
    oibdp,             -- Operating Income Before Depreciation
    ebit,              -- Earnings Before Interest and Taxes
    sale,              -- Sales/Revenue
    cogs,              -- Cost of Goods Sold
    ni,                -- Net Income
    oancf,             -- Operating Cash Flow
    fincf,             -- Financing Cash Flow
    csho,              -- Common Shares Outstanding
    prcc_f,            -- Price Close
    'Annual' AS freq   -- Data Frequency Indicator
FROM 
    comp.funda
WHERE  
    indfmt = 'INDL'     -- Industrial Format
    AND datafmt = 'STD' -- Standardized Format
    AND popsrc = 'D'     -- Domestic Companies
    AND consol = 'C'     -- Consolidated Data

    AND fyear >= 1990    -- Adjust the starting fiscal year as needed
"""

financials = conn.raw_sql(sql_financials)

print(f'\nThe number of rows is {len(financials):,}.\n')
financials.head(10)


The number of rows is 396,714.



,gvkey,datadate,fyear,fyr,at,lt,ceq,act,lct,invt,...,oibdp,ebit,sale,cogs,ni,oancf,fincf,csho,prcc_f,freq
0,001004,1991-05-31,1990,5,379.958,186.18,193.778,268.399,79.227,156.133,...,41.957,33.701,466.542,366.04,14.801,36.891,-32.043,15.891,14.124999,Annual
1,001009,1990-10-31,1990,10,32.335,26.073,6.262,10.047,8.382,2.355,...,6.254,4.283,40.522,30.633,2.135,2.075,3.4,2.526,5.999998,Annual
2,001010,1990-12-31,1990,12,1728.888,1516.287,210.399,523.502,421.777,36.86,...,177.87,121.123,312.309,106.729,-75.287,28.377,83.508,0.015,<NA>,Annual
3,001011,1990-12-31,1990,12,7.784,7.117,0.667,1.247,1.246,0.663,...,-0.57,-1.27,3.635,1.125,-1.869,-0.781,2.001,3.556,1.188,Annual
4,001013,1990-10-31,1990,10,181.665,47.652,134.013,102.525,37.335,33.845,...,49.725,37.745,259.802,121.822,22.903,34.695,-0.312,13.289,16.375,Annual
5,001014,1990-06-30,1990,6,13.491,6.076,7.415,5.025,2.739,0.0,...,1.475,0.701,2.644,1.169,2.037,0.677,-0.123,0.685,11.0,Annual
6,001017,1991-02-28,1990,2,118.12,73.727,44.393,55.04,32.615,4.292,...,12.892,6.621,144.258,107.343,2.574,9.994,-3.112,3.882,5.125,Annual
7,001019,1990-12-31,1990,12,21.716,13.293,8.423,9.075,3.001,0.284,...,4.649,2.692,22.667,12.09,1.721,4.083,-2.219,0.206,<NA>,Annual
8,001021,1990-06-30,1990,6,34.591,25.545,7.751,24.684,16.208,15.463,...,4.47,3.145,48.292,31.214,0.422,0.865,-0.655,4.755,1.561999,Annual
9,001025,1991-02-28,1990,2,5.631,2.832,2.799,5.323,2.704,2.798,...,0.614,0.573,11.285,7.111,0.126,0.549,-0.289,4.207,<NA>,Annual


In [ ]:
# sort the financial data
financials = financials.sort_values(['gvkey', 'datadate'])
# financials.to_csv('financials.csv', index=False)

In [17]:
# this is where we get information on industry and region

sql_info = """
SELECT 
    gvkey,
    conm,       -- Company Name
    fic,        -- Current ISO Country Code - Incorporation
    fyrc,       -- month of fiscal year end
    gsector,    -- GIC Sector  (Global Identification Code)
    ggroup,     -- GIC Group
    gind,       -- GIC Industries
    idbflag,    -- International, Domestic, Both Indicator
    incorp,     -- Current State/Province of Incorporation Code
    loc,        -- Current ISO Country Code - Headquarters
    naics,      -- North American Industry Classification Code
    sic,        -- Standard Identification Code
    state       -- State/Province
FROM 
    comp.company
"""
# Note that GICs is the current industry leader while SIC and NAICs are considered outdated

info = conn.raw_sql(sql_info)

print(f'\nThe number of rows is {len(info):,}.\n')
info.head()


The number of rows is 54,885.



,gvkey,conm,fic,fyrc,gsector,ggroup,gind,idbflag,incorp,loc,naics,sic,state
0,001000,A & E PLASTIK PAK INC,USA,12,<NA>,<NA>,<NA>,D,<NA>,USA,<NA>,3089,<NA>
1,001001,A & M FOOD SERVICES INC,USA,12,25,2530,253010,D,<NA>,USA,722,5812,OK
2,001002,AAI CORP,USA,12,<NA>,<NA>,<NA>,D,<NA>,USA,<NA>,3825,MD
3,001003,A.A. IMPORTING CO INC,USA,1,25,2550,255040,D,DE,USA,442110,5712,MO
4,001004,AAR CORP,USA,5,20,2010,201010,D,DE,USA,423860,5080,IL


- For [GICS](https://www.msci.com/indexes/documents/methodology/0_MSCI_Global_Industry_Classification_Standard_GICS_Methodology_20240801.pdf): gsector = 40, where banks are 4010, diversified financials are 4020, insurance 4030; real estate gsector 60
- For SIC standard: SIC codes 6000-6799 | Chava & Jarrow (2004)
- For [NAICS](https://www.naics.com/naics-code-description/?code=52), finance and insurance is 52

In [23]:
# Define filtered_info from info DataFrame
filtered_info = info.copy()

# 确保 sic 和 naics 是数值型，避免数据类型错误
filtered_info['sic'] = pd.to_numeric(filtered_info['sic'], errors='coerce')
filtered_info['naics'] = pd.to_numeric(filtered_info['naics'], errors='coerce')

# we prefer GICS over SIC and NAICS
filtered_info = filtered_info[~filtered_info['gsector'].isin([40, 60])].copy()

# if there is no GICS, use SIC
mask_no_gics = filtered_info['gsector'].isna()
mask_sic = filtered_info['sic'].between(6000, 6799)
filtered_info = filtered_info.loc[~(mask_no_gics & mask_sic)].copy()

# if there is no GICS and SIC, use NAICS(redundant)
mask_no_gics_sic = mask_no_gics & filtered_info['sic'].isna()
mask_naics = filtered_info.loc[mask_no_gics_sic, 'naics'].between(520000, 529999)

filtered_info


,gvkey,conm,fic,fyrc,gsector,ggroup,gind,idbflag,incorp,loc,naics,sic,state
0,001000,A & E PLASTIK PAK INC,USA,12,<NA>,<NA>,<NA>,D,<NA>,USA,<NA>,3089,<NA>
1,001001,A & M FOOD SERVICES INC,USA,12,25,2530,253010,D,<NA>,USA,722,5812,OK
2,001002,AAI CORP,USA,12,<NA>,<NA>,<NA>,D,<NA>,USA,<NA>,3825,MD
3,001003,A.A. IMPORTING CO INC,USA,1,25,2550,255040,D,DE,USA,442110,5712,MO
4,001004,AAR CORP,USA,5,20,2010,201010,D,DE,USA,423860,5080,IL
...,...,...,...,...,...,...,...,...,...,...,...,...,...
54880,355398,SUPER HI INTERNATIONAL HOLDI,CYM,12,25,2530,253010,B,<NA>,SGP,722511,5812,<NA>
54881,356128,JOINT STOCK COMPANY KASPI KZ,KAZ,12,40,4020,402020,B,<NA>,KAZ,522291,6141,<NA>
54882,356289,SOLARBANK CORP,CAN,6,20,2010,201030,D,<NA>,CAN,2371,1600,ON
54883,356687,ATHA ENERGY CORP,CAN,12,10,1010,101020,D,<NA>,CAN,212290,1090,BC


In [24]:
# check if there are any companies with no industry information
no_industry_info = filtered_info[filtered_info[['gsector', 'sic', 'naics']].isna().all(axis=1)]

if not no_industry_info.empty:
    print("\nCompanies with no industry classification:")
    no_industry_info

In [27]:
industrial_financials = financials.merge(filtered_info, on='gvkey', how='inner')
# industrial_financials_check = financials.merge(filtered_info, on='gvkey', how='right')
print(f'\nThe number of rows in industrial_financials: {len(industrial_financials):,}.\n')
# print(f'\nThe number of rows in industrial_financials_check: {len(industrial_financials_check):,}.\n')


The number of rows in industrial_financials: 335,645.



There typically some ways I can figure out to treat the data.

1. Use both the `fyear` and `fyr`. When conducting an regression, add 11 dummy variables to introduce individual fixed effects and time fixed effects.
2. Set a date, for example 6-30(because all company should provide their annual report right now if their fiscal year ends at 6-30) as the end of a year, and use cloest avaiable data to represent the data of the year.
3. Regardless of the `fyr`, according to `ff3_v3_dwd.ipynb`, just use the year. But we will keep the datadate in our case to calculate the default flag.
4. Calendarization the fisical data.

I use 3.

In [ ]:
# industrial_financials.to_csv('industrial_financials.csv', index=False)

## Merge the two data set

- Merge `ratings` and `industrail_financials`
    - what we want is table consists of both ratings (for the year) and financial datas (for the `fyear`)
    - if there are multiple ratings in a year, use the closest one to the 12-31
    - there could be other ways to conduct this

In [41]:
print(ratings.head())

    gvkey  companyid  startdate    enddate  entity_pname ratingdate  \
0  210835    18511.0 1901-01-01 2261-12-31  3i Group PLC 1995-12-07   
2  210835    18511.0 1901-01-01 2261-12-31  3i Group PLC 2001-12-20   
5  210835    18511.0 1901-01-01 2261-12-31  3i Group PLC 2008-11-11   
6  210835    18511.0 1901-01-01 2261-12-31  3i Group PLC 2009-01-28   
7  210835    18511.0 1901-01-01 2261-12-31  3i Group PLC 2009-02-23   

  ratingsymbol ratingactionword unsol  dflt_date  
0          AA-       New Rating     N 2261-12-31  
2           A+        Downgrade     N 2261-12-31  
5            A        Downgrade     N 2261-12-31  
6           A-        Downgrade     N 2261-12-31  
7         BBB+        Downgrade     N 2261-12-31  


In [47]:
# record default date

ratings.loc[:,'dflt_date'] = pd.Timestamp("2261-12-31")

dflt_dates = ratings.loc[ratings['ratingsymbol'].isin(['D', 'SD', 'R']), ['gvkey', 'ratingdate']]
dflt_dates = dflt_dates.groupby('gvkey')['ratingdate'].min().reset_index()
ratings = ratings.merge(dflt_dates, on='gvkey', how='left', suffixes=('', '_dflt'))
ratings['dflt_date'] = ratings['ratingdate_dflt'].fillna(ratings['dflt_date'])
ratings.drop(columns=['ratingdate_dflt'], inplace=True)

# check for "001045"
ratings.loc[ratings['gvkey'] == '001045']


,gvkey,companyid,startdate,enddate,entity_pname,ratingdate,ratingsymbol,ratingactionword,unsol,dflt_date
7656,001045,168569.0,1901-01-01,2261-12-31,American Airlines Group Inc.,1990-08-20,A-,Downgrade,N,2011-11-29
7657,001045,168569.0,1901-01-01,2261-12-31,American Airlines Group Inc.,1991-04-05,BBB+,Downgrade,N,2011-11-29
7658,001045,168569.0,1901-01-01,2261-12-31,American Airlines Group Inc.,1992-06-01,BBB,Downgrade,N,2011-11-29
7659,001045,168569.0,1901-01-01,2261-12-31,American Airlines Group Inc.,1993-03-11,BB+,<NA>,N,2011-11-29
7660,001045,168569.0,1901-01-01,2261-12-31,American Airlines Group Inc.,1997-06-16,BBB-,<NA>,N,2011-11-29
7661,001045,168569.0,1901-01-01,2261-12-31,American Airlines Group Inc.,2001-09-20,BB,Downgrade,N,2011-11-29
7662,001045,168569.0,1901-01-01,2261-12-31,American Airlines Group Inc.,2002-06-28,BB-,Downgrade,N,2011-11-29
7663,001045,168569.0,1901-01-01,2261-12-31,American Airlines Group Inc.,2003-02-28,B-,Downgrade,N,2011-11-29
7664,001045,168569.0,1901-01-01,2261-12-31,American Airlines Group Inc.,2003-03-28,CCC,Downgrade,N,2011-11-29
7665,001045,168569.0,1901-01-01,2261-12-31,American Airlines Group Inc.,2003-06-20,B-,Upgrade,N,2011-11-29


In [ ]:
# change the ratings to anual frequency, align with financials

ratings['year'] = ratings['ratingdate'].dt.year

ratings_sorted = ratings.sort_values(by=['gvkey', 'year', 'ratingdate'])
ratings_last_per_year = ratings_sorted.groupby(['gvkey', 'year']).last().reset_index()
default_years = ratings[ratings['ratingsymbol'].isin(['D', 'SD', 'R'])].groupby(['gvkey', 'year']).first().reset_index()
ratings_annual = ratings_last_per_year.merge(default_years[['gvkey', 'year', 'ratingsymbol']], 
                                             on=['gvkey', 'year'], 
                                             how='left', 
                                             suffixes=('', '_default'))
ratings_annual['ratingsymbol'] = ratings_annual['ratingsymbol_default'].fillna(ratings_annual['ratingsymbol'])

ratings_annual = ratings_annual[['year', 'gvkey', 'companyid', 'entity_pname', 'ratingsymbol', 'dflt_date']]

# expand the ratings to all years
def expand_years(group):
    first_rating_year = group['year'].min()
    all_years = pd.DataFrame({'year': range(first_rating_year, 2025)})
    all_years['gvkey'] = group['gvkey'].iloc[0]
    return all_years.merge(group, on=['gvkey', 'year'], how='left')

ratings_full = ratings_annual.groupby('gvkey', group_keys=False).apply(expand_years)

ratings_full = ratings_full.reset_index(drop=True)

# fill all columns with the last known value
fill_cols = ['ratingsymbol', 'companyid', 'entity_pname', 'dflt_date']

def custom_ffill(group):
    mask = group['ratingsymbol'].isin(['D', 'SD', 'R'])
    group[fill_cols] = group[fill_cols].ffill()
    group.loc[mask, 'ratingsymbol'] = group.loc[mask, 'ratingsymbol']
    return group

ratings_full = ratings_full.groupby('gvkey', group_keys=False).apply(custom_ffill)
ratings_full = ratings_full.sort_values(by=['gvkey', 'year'])

C:\Users\123\AppData\Local\Temp\ipykernel_17600\2969734725.py:34: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ratings_full = ratings_annual.groupby('gvkey', group_keys=False).apply(expand_years)
C:\Users\123\AppData\Local\Temp\ipykernel_17600\2969734725.py:48: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ratings_full = ratings_full.groupby('gvkey', group_keys=False).apply(custom_ffill)


In [59]:
ratings_full.head(10)

,year,gvkey,companyid,entity_pname,ratingsymbol,dflt_date
0,1995,001004,168154.0,AAR Corp.,BBB-,2261-12-31
1,1996,001004,168154.0,AAR Corp.,BBB-,2261-12-31
2,1997,001004,168154.0,AAR Corp.,BBB,2261-12-31
3,1998,001004,168154.0,AAR Corp.,BBB,2261-12-31
4,1999,001004,168154.0,AAR Corp.,BBB,2261-12-31
5,2000,001004,168154.0,AAR Corp.,BBB,2261-12-31
6,2001,001004,168154.0,AAR Corp.,BBB,2261-12-31
7,2002,001004,168154.0,AAR Corp.,BBB-,2261-12-31
8,2003,001004,168154.0,AAR Corp.,BB-,2261-12-31
9,2004,001004,168154.0,AAR Corp.,BB-,2261-12-31


In [61]:
# check for "001045"
ratings_full.loc[ratings_full['gvkey'] == '001045']

,year,gvkey,companyid,entity_pname,ratingsymbol,dflt_date
73,1990,001045,168569.0,American Airlines Group Inc.,A-,2011-11-29
74,1991,001045,168569.0,American Airlines Group Inc.,BBB+,2011-11-29
75,1992,001045,168569.0,American Airlines Group Inc.,BBB,2011-11-29
76,1993,001045,168569.0,American Airlines Group Inc.,BB+,2011-11-29
77,1994,001045,168569.0,American Airlines Group Inc.,BB+,2011-11-29
78,1995,001045,168569.0,American Airlines Group Inc.,BB+,2011-11-29
79,1996,001045,168569.0,American Airlines Group Inc.,BB+,2011-11-29
80,1997,001045,168569.0,American Airlines Group Inc.,BBB-,2011-11-29
81,1998,001045,168569.0,American Airlines Group Inc.,BBB-,2011-11-29
82,1999,001045,168569.0,American Airlines Group Inc.,BBB-,2011-11-29


In [82]:
industrial_financials = industrial_financials.rename(columns={'fyear': 'year'})
merged_data = ratings_full.merge(industrial_financials, on=['gvkey', 'year'], how='inner')
merged_data.tail(20)

,year,gvkey,companyid,entity_pname,ratingsymbol,dflt_date,datadate,fyr,at,lt,...,fyrc,gsector,ggroup,gind,idbflag,incorp,loc,naics,sic,state
60126,2020,326688,542155250.0,nVent Electric PLC,BBB-,2261-12-31,2020-12-31,12,4366.1,1956.3,...,12,20,2010,201040,D,<NA>,GBR,3323,3440,<NA>
60127,2021,326688,542155250.0,nVent Electric PLC,BBB-,2261-12-31,2021-12-31,12,4674.2,2178.1,...,12,20,2010,201040,D,<NA>,GBR,3323,3440,<NA>
60128,2022,326688,542155250.0,nVent Electric PLC,BBB-,2261-12-31,2022-12-31,12,4902.2,2170.5,...,12,20,2010,201040,D,<NA>,GBR,3323,3440,<NA>
60129,2023,326688,542155250.0,nVent Electric PLC,BBB-,2261-12-31,2023-12-31,12,6161.7,3019.6,...,12,20,2010,201040,D,<NA>,GBR,3323,3440,<NA>
60130,2024,326688,542155250.0,nVent Electric PLC,BBB-,2261-12-31,2024-12-31,12,6734.9,3497.3,...,12,20,2010,201040,D,<NA>,GBR,3323,3440,<NA>
60131,2021,327677,82871890.0,Marex Group PLC,BBB-,2261-12-31,2021-12-31,12,<NA>,<NA>,...,12,40,4020,402030,B,<NA>,GBR,523,6200,<NA>
60132,2022,327677,82871890.0,Marex Group PLC,BBB-,2261-12-31,2022-12-31,12,15743.9,15066.2,...,12,40,4020,402030,B,<NA>,GBR,523,6200,<NA>
60133,2023,327677,82871890.0,Marex Group PLC,BBB-,2261-12-31,2023-12-31,12,17750.1,16974.2,...,12,40,4020,402030,B,<NA>,GBR,523,6200,<NA>
60134,2021,328795,546410010.0,Arcosa Inc.,BB,2261-12-31,2021-12-31,12,3188.1,1234.8,...,12,20,2010,201030,D,DE,USA,332312,3440,TX
60135,2022,328795,546410010.0,Arcosa Inc.,BB,2261-12-31,2022-12-31,12,3340.6,1156.2,...,12,20,2010,201030,D,DE,USA,332312,3440,TX


In [83]:
# 328920
with pd.option_context('display.max_columns', None):
    print(merged_data.loc[merged_data['gvkey'] == '328920'])

       year   gvkey    companyid                    entity_pname ratingsymbol  \
60137  2020  328920  582534656.0  Coronado Global Resources Inc.            B   
60138  2021  328920  582534656.0  Coronado Global Resources Inc.            B   
60139  2022  328920  582534656.0  Coronado Global Resources Inc.           B+   
60140  2023  328920  582534656.0  Coronado Global Resources Inc.           B+   

       dflt_date    datadate  fyr        at        lt       ceq      act  \
60137 2261-12-31  2020-12-31   12      <NA>      <NA>      <NA>     <NA>   
60138 2261-12-31  2021-12-31   12      <NA>      <NA>      <NA>     <NA>   
60139 2261-12-31  2022-12-31   12  2549.927  1446.837   1103.09  989.028   
60140 2261-12-31  2023-12-31   12  2678.038  1432.003  1246.035   944.04   

           lct     invt     rect       ap     dlc     dltt  dltis      dvt  \
60137     <NA>     <NA>     <NA>     <NA>    <NA>     <NA>   <NA>     <NA>   
60138     <NA>     <NA>     <NA>     <NA>    <NA>     <NA

In [84]:
cols_to_drop = ['gsector', 'ggroup', 'gind', 'idbflag', 'incorp', 'loc', 'naics', 'sic', 'state', 'freq', 'fic']
merged_data = merged_data.drop(columns=cols_to_drop, errors='ignore')

financial_cols = ['at', 'lt', 'ceq', 'sale', 'ni', 'oibdp']
merged_data = merged_data.dropna(subset=financial_cols, how='all')

with pd.option_context('display.max_columns', None):
    print(merged_data.loc[merged_data['gvkey'] == '328920'])

       year   gvkey    companyid                    entity_pname ratingsymbol  \
60139  2022  328920  582534656.0  Coronado Global Resources Inc.           B+   
60140  2023  328920  582534656.0  Coronado Global Resources Inc.           B+   

       dflt_date    datadate  fyr        at        lt       ceq      act  \
60139 2261-12-31  2022-12-31   12  2549.927  1446.837   1103.09  989.028   
60140 2261-12-31  2023-12-31   12  2678.038  1432.003  1246.035   944.04   

           lct     invt     rect       ap     dlc     dltt  dltis      dvt  \
60139   600.86  158.018  422.046    61.78  12.178  256.794    0.0  701.655   
60140  507.725  192.279   320.56  113.273  25.704  302.342    0.0   16.765   

           che    xint   xrd   xsga     oibdp     ebit      sale      cogs  \
60139  334.629  69.132  <NA>  46.32  1207.953  1042.45  3571.542  2317.269   
60140    361.2  64.351  <NA>  38.64   381.369  217.507  2890.603  2470.594   

            ni    oancf    fincf      csho  prcc_f  \
601

### Build Ratios

In [85]:
available_columns = merged_data.columns.tolist()
print(available_columns)

['year', 'gvkey', 'companyid', 'entity_pname', 'ratingsymbol', 'dflt_date', 'datadate', 'fyr', 'at', 'lt', 'ceq', 'act', 'lct', 'invt', 'rect', 'ap', 'dlc', 'dltt', 'dltis', 'dvt', 'che', 'xint', 'xrd', 'xsga', 'oibdp', 'ebit', 'sale', 'cogs', 'ni', 'oancf', 'fincf', 'csho', 'prcc_f', 'conm', 'fyrc']


In [104]:
import numpy as np

information_columns = ['gvkey', 'year', 'entity_pname']

financial_columns = [
    'at', 'lt', 'ceq', 'act', 'lct', 'invt', 'rect', 'ap',
    'dlc', 'dltt', 'che', 'sale', 'cogs', 'ni', 'ebit', 'oibdp', 'xint',
    'xsga', 'oancf', 'csho', 'prcc_f'
]

default_columns = ['ratingsymbol', 'dflt_date']

merged_data = merged_data[information_columns + financial_columns + default_columns]

merged_data['ACTLCT'] = merged_data['act'] / merged_data['lct']
merged_data['APSALE'] = merged_data['ap'] / merged_data['sale']
merged_data['CASHTA'] = merged_data['che'] / merged_data['at']
merged_data['CHAT'] = merged_data['che'] / merged_data['at']
merged_data['CHLCT'] = merged_data['che'] / merged_data['lct']
merged_data['EBITAT'] = merged_data['ebit'] / merged_data['at']
merged_data['EBITSALE'] = merged_data['ebit'] / merged_data['sale']
merged_data['FAT'] = (merged_data['dlc'] + 0.5 * merged_data['dltt']) / merged_data['at']
merged_data['FFOLT'] = merged_data['oancf'] / merged_data['lt']
merged_data['INVTSALES'] = merged_data['invt'] / merged_data['sale']
merged_data['LCTLT'] = merged_data['lct'] / merged_data['lt']
merged_data['LOGAT'] = merged_data['at'].apply(lambda x: None if x <= 0 else np.log(x))
merged_data['LOGSALE'] = merged_data['sale'].apply(lambda x: None if x <= 0 else np.log(x))
merged_data['NIAT'] = merged_data['ni'] / merged_data['at']
merged_data['NIMTA'] = merged_data['ni'] / (merged_data['prcc_f'] * merged_data['csho'] + merged_data['lt'])
merged_data['NISALE'] = merged_data['ni'] / merged_data['sale']

ordered_columns = information_columns + default_columns + financial_columns +[
    'ACTLCT', 'APSALE', 'CASHTA', 'CHAT', 'CHLCT', 'EBITAT', 'EBITSALE',
    'FAT', 'FFOLT', 'INVTSALES', 'LCTLT', 'LOGAT', 'LOGSALE', 'NIAT',
    'NIMTA', 'NISALE'
]

df = merged_data[ordered_columns]


In [106]:
# Calculate the missing data count per column and the missing percentage
missing_counts = df.isnull().sum()                   # Count missing values per column
missing_percentage = 100 * df.isnull().sum() / df.shape[0]  # Calculate missing percentage per column

# Create a summary DataFrame with missing counts and percentages
missing_summary = pd.DataFrame({
    'Missing Count': missing_counts,
    'Missing Percentage (%)': missing_percentage
})

# Print the summary to the console
print("Missing Data Summary:")
with pd.option_context('display.max_rows', None):    
    print(missing_summary)

Missing Data Summary:
              Missing Count  Missing Percentage (%)
gvkey                     0                0.000000
year                      0                0.000000
entity_pname              0                0.000000
ratingsymbol              0                0.000000
dflt_date                 0                0.000000
at                       13                0.021870
lt                       50                0.084117
ceq                     223                0.375162
act                   11800               19.851618
lct                   11751               19.769183
invt                   1334                2.244242
rect                    556                0.935381
ap                     1181                1.986844
dlc                     188                0.316280
dltt                     88                0.148046
che                      44                0.074023
sale                     50                0.084117
cogs                     61               

In [107]:
df.to_csv('merged_data.csv', index=False)

### Default Flag

In [124]:
df = df.copy()

df['year_end'] = pd.to_datetime(df['year'], format='%Y') + pd.DateOffset(days=364)
df['daystod'] = (df['dflt_date'] - df['year_end']).dt.days
df['default_flag'] = 0
df.loc[(df['daystod'] >= 90) & (df['daystod'] <= 455), 'default_flag'] = 1

ordered_columns = information_columns + default_columns + ['daystod', 'default_flag'] + financial_columns +[
    'ACTLCT', 'APSALE', 'CASHTA', 'CHAT', 'CHLCT', 'EBITAT', 'EBITSALE',
    'FAT', 'FFOLT', 'INVTSALES', 'LCTLT', 'LOGAT', 'LOGSALE', 'NIAT',
    'NIMTA', 'NISALE'
]

df = df[ordered_columns]
df

,gvkey,year,entity_pname,ratingsymbol,dflt_date,daystod,default_flag,at,lt,ceq,...,EBITSALE,FAT,FFOLT,INVTSALES,LCTLT,LOGAT,LOGSALE,NIAT,NIMTA,NISALE
0,001004,1995,AAR Corp.,BBB-,2261-12-31,97155,0,437.846,233.211,204.635,...,0.064243,0.138451,0.10617,0.346708,0.3404,6.081867,6.224539,0.03657,0.02727,0.031708
1,001004,1996,AAR Corp.,BBB-,2261-12-31,96790,0,529.584,260.325,269.259,...,0.072778,0.113076,0.036612,0.368622,0.384062,6.272092,6.378983,0.043478,0.027921,0.03907
2,001004,1997,AAR Corp.,BBB,2261-12-31,96424,0,670.559,369.709,300.85,...,0.082744,0.132712,0.061732,0.336808,0.40342,6.508112,6.662012,0.053175,0.032353,0.04559
3,001004,1998,AAR Corp.,BBB,2261-12-31,96059,0,726.63,400.595,326.035,...,0.08429,0.125084,0.071207,0.331685,0.43332,6.588417,6.822237,0.057348,0.044266,0.045391
4,001004,1999,AAR Corp.,BBB,2261-12-31,95694,0,740.998,401.483,339.515,...,0.06898,0.157271,0.025035,0.328036,0.408027,6.607998,6.931797,0.047454,0.045416,0.034328
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60141,351590,2021,Daimler Truck Holding Ag,BBB+,2261-12-31,87658,0,62325.449,43647.149,18106.225,...,0.04147,0.201442,0.05472,0.195981,0.451937,11.040125,10.719397,0.042828,0.036074,0.059023
60142,351590,2022,Daimler Truck Holding Ag,BBB+,2261-12-31,87293,0,68366.372,46343.87,21430.419,...,0.068683,0.221592,-0.012061,0.173428,0.483753,11.132636,10.902685,0.041661,0.039687,0.052432
60143,351590,2023,Daimler Truck Holding Ag,BBB+,2261-12-31,86928,0,78713.854,54148.658,23880.987,...,0.09032,0.241027,0.007879,0.163804,0.465767,11.273574,11.031299,0.053011,0.049433,0.067543
60144,353444,2022,Haleon PLC,BBB,2261-12-31,87293,0,41948.594,22119.555,19677.222,...,0.222693,0.156211,0.112376,0.124148,0.238043,10.644200,9.479054,0.030447,0.021626,0.097624


In [125]:
df.loc[df['gvkey'] == '001045']

,gvkey,year,entity_pname,ratingsymbol,dflt_date,daystod,default_flag,at,lt,ceq,...,EBITSALE,FAT,FFOLT,INVTSALES,LCTLT,LOGAT,LOGSALE,NIAT,NIMTA,NISALE
31,001045,1990,American Airlines Group Inc.,A-,2011-11-29,7638,0,13353.6,9626.2,3727.4,...,0.010581,0.221292,0.071243,0.051922,0.501205,9.499541,9.369018,-0.002965,-0.003133,-0.003379
32,001045,1991,American Airlines Group Inc.,BBB+,2011-11-29,7273,0,16208.0,12414.0,3794.0,...,0.000388,0.24528,0.059932,0.04811,0.381988,9.693260,9.463974,-0.014808,-0.013947,-0.018623
33,001045,1992,American Airlines Group Inc.,BBB,2011-11-29,6908,0,18706.0,15357.0,3349.0,...,-0.001737,0.244895,0.054894,0.047583,0.307352,9.836600,9.574706,-0.049984,-0.045729,-0.064949
34,001045,1993,American Airlines Group Inc.,BB+,2011-11-29,6542,0,19326.0,15050.0,3195.0,...,0.036622,0.211477,0.091495,0.043819,0.293488,9.869207,9.661480,-0.005692,-0.005465,-0.007006
35,001045,1994,American Airlines Group Inc.,BB+,2011-11-29,6177,0,19486.0,16106.0,3302.0,...,0.079569,0.238992,0.099901,0.042015,0.305104,9.877452,9.688870,0.011701,0.011316,0.014129
36,001045,1995,American Airlines Group Inc.,BB+,2011-11-29,5812,0,19556.0,15836.0,3642.0,...,0.091543,0.1982,0.137977,0.034831,0.29635,9.881037,9.735660,0.00854,0.007764,0.009876
37,001045,1996,American Airlines Group Inc.,BB+,2011-11-29,5447,0,20497.0,14829.0,5668.0,...,0.105053,0.137825,0.183155,0.035656,0.375346,9.928034,9.784310,0.049568,0.044467,0.05723
38,001045,1997,American Airlines Group Inc.,BBB-,2011-11-29,5081,0,20915.0,14699.0,6216.0,...,0.103716,0.118408,0.198653,0.034249,0.382135,9.948222,9.829303,0.047095,0.038138,0.053043
39,001045,1998,American Airlines Group Inc.,BBB-,2011-11-29,4716,0,22303.0,15605.0,6698.0,...,0.121739,0.103215,0.204742,0.031034,0.361359,10.012476,9.862926,0.058916,0.052173,0.06842
40,001045,1999,American Airlines Group Inc.,BBB-,2011-11-29,4351,0,24374.0,17516.0,6858.0,...,0.0652,0.138775,0.129253,0.039932,0.33478,10.101272,9.783013,0.040412,0.035885,0.055556
